# Histogram obrazu. Wyrównywanie histogramu.

## Cel ćwiczenia

- Zapoznanie z pojęciem histogramu obrazu (w odcieniach szarości i kolorze).
- Zapoznanie z metodami modyfikacji histogramu (rozciąganie, wyrównywanie, dopasowywanie).

## Histogram

- Histogramem obrazu nazywamy wykres słupkowy zdefiniowany następującymi zależnościami:<br>
\begin{equation}
h(i) = \sum_{x=0}^{N-1} \sum_{y=0}^{M-1} p(i,(x,y))
\end{equation}<br>
gdzie:<br>
\begin{equation}
p(i) =  \left\{
  \begin{array}{l l}
    1 & \quad \text{gdy} f(x,y) = i\\
    0 & \quad \text{gdy} f(x,y) \ne i
  \end{array} \right.
\end{equation}

- Inaczej mówiąc, histogram zawiera informacje na temat tego ile pikseli o danym poziomie jasności występuje na obrazie (w przypadku obrazu w odcieniach szarości). Określa się to także rozkładem empirycznym cechy.

- Często wykorzystuje się tzw. znormalizowaną postać histogramu  – wszystkie wartości $h(i)$ są dzielone przez liczbę pikseli na obrazie.
Otrzymana w ten sposób wielkość to gęstość prawdopodobieństwa wystąpienia na obrazie pikseli o odcieniu $i$.

- Histogram można zdefiniować również dla obrazów kolorowych.
Otrzymujemy wtedy 3 histogramy – po jednym dla danej składowej: R,G,B (lub HSV, YCbCr, itp.) lub histogram trójwymiarowy.

- Histogram jest bardzo użyteczny w przetwarzaniu i analizie obrazów.
Wykorzystywany jest przy binaryzacji (szerzej na jednym z kolejnych laboratoriów) oraz do oceny jakości (dynamiki, kontrastu) obrazu.
W idealnym przypadku wszystkie poziomy jasności w obrazie powinny być wykorzystane (i to najlepiej w miarę jednolicie)  – obrazowo mówiąc histogram powinien rozciągać się od 0  – 255 (obraz w skali szarości).

- W przypadku gdy  wykorzystujemy jedynie fragment dostępnego zakresu (wąski histogram)  lub histogram nie jest jednolity (występują dominujące grupy pikseli) obraz ma dość słaby kontrast.
Cechę tę można poprawić stosując tzw. rozciąganie albo wyrównywanie histogramu (ang. *histogram equalization*).</div>

## Histogram dla obrazów w odcieniach szarości

1. Zaimportuj potrzebne biblioteki: *OpenCV*, *pyplot* z *matplotlib* i *numpy*.
        import cv2
        from matplotlib import pyplot as plt
        import numpy as np
2. Wczytaj obrazy *lenaX.bmp* w skali szarości. *X* jest numerem wczytywanego obrazu (1 - 4).
        I = cv2.imread('lenaX.bmp', cv2.IMREAD_GRAYSCALE)
3. Oblicz histogram wczytanego obrazu wykorzystując funkcję `cv2.calcHist`.
    - Pierwszym argumentem jest obraz, dla którego obliczony zostanie histogram.
    Należy go przekazać w nawiasie kwadratowym.
    - Drugim argumentem jest numer kanału, dla którego ma zostać obliczony histogram.
    Również powinien być przekazany w nawiasie kwadratowym.
    - Trzeci argument oznacza maskę, czyli obszar, dla którego ma zostać wyznaczony histogram.
    Aby obliczyć dla całego obrazu należy przekazać *None*.
    - Czwartym argumentem jest rozmiar histogramu (liczba przedziałów).
    Argument powinien być w nawiasie kwadratowym. Dla pełnej skali należy przekazać wartość *256*.
    - Ostatnim argumentem jest zakres wartości. Dla obrazów typu *uint8* powinien on wynosić *[0, 256]*.
    - Funkcja zwraca obliczony histogram.
4. Wyświetl wczytane obrazy i ich histogramy w jednym oknie. Użyj `plt.subplot()` w celu stworzenia siatki wykresów.
        figLena, axsLena = plt.subplots(2, 4)
Rozmiar utworzonego okna można zmienić wykorzystując instrukcję (uwaga w calach -  1 cal to 2.54cm):
        figLena.set_size_inches(20, 10)
Przykładowe wyświetlenie obrazu:
        axsLena[0, 0].imshow(I1, 'gray', vmin=0, vmax=256)
        axsLena[0, 0].axis('off')
Przykładowe wyświetlenie histogramu:
        axsLena[1, 0].plot(H1)
        axsLena[1, 0].grid()
5. Przeanalizuj (dokładnie) związek histogramu z jasnością i ostrością obrazu (tu rozumianą jako subiektywne odczucie).

In [ ]:
import cv2
import os
from matplotlib import pyplot as plt
import numpy as np

if not os.path.exists("lena1.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/03_Histogram/lena1.bmp --no-check-certificate

if not os.path.exists("lena2.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/03_Histogram/lena2.bmp --no-check-certificate

if not os.path.exists("lena3.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/03_Histogram/lena3.bmp --no-check-certificate

if not os.path.exists("lena4.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/03_Histogram/lena4.bmp --no-check-certificate
img = [cv2.imread(f'lena{i}.bmp', cv2.IMREAD_GRAYSCALE) for i in range(1,5)]
plt.gray()
fig, ax = plt.subplots(2,4)
fig.set_size_inches(18.5, 10.5)
for n,j in enumerate(img):
    ax[n//2][2*(n%2)].imshow(j)
    ax[n//2][2*(n%2)].set_xticks([])
    ax[n//2][2*(n%2)].set_yticks([])
    ax[n//2][2*(n%2)].set_title(f"lena{n+1}.bmp")
    ax[n//2][2*(n%2) + 1].plot(cv2.calcHist([j],[0], None, [256] ,[0,255]))
    ax[n//2][2*(n%2) + 1].set_title(f"lena{n+1}.bmp histogram")
plt.show()


## Rozciąganie histogramu

Najprostszą metodą poprawienia jakości obrazu jest tzw. rozciągnięcie histogramu.
Polega na przeskalowaniu wartości pikseli w obrazie tak, aby wykorzystać cały dostępny zakres [0-255] (oczywiście w przypadku obrazów w odcieniach szarości w reprezentacji 8-bitowej).

1. Wczytaj obraz *hist1.bmp* w skali szarości.
Oblicz i wyświetl histogram rozpatrywanego obrazu (na wspólnym rysunku z obrazem).
Zwróć uwagę na ilość widocznych szczegółów.
2. Rozciągnij histogram obrazu. W tym celu można wykorzystać funkcję `cv2.normalize`.
    - Pierwszym argumentem funkcji jest obraz poddawany operacji.
    - Drugim argumentem jest tablica do której zostanie wpisany wynik.
    Należy ją najpierw zainicjalizować.
    Najlepiej zrobić to funkcją `np.zeros`, której pierwszym argumentem jest rozmiar obrazu (`I.shape`), a drugim typ danych (`uint8`).
    Można również przekazać `None`, a wynik przypisać do nowej zmiennej.
    - Trzecim argumentem jest minimalna wartość po normalizacji.
    - Czwartym argumentem jest wartość maksymalna po normalizacji.
    - Ostatnim argumentem jest typ wykorzystanej normy (uogólnienie pojęcia długości wektora).
    Należy wykorzystać normę `cv2.NORM_MINMAX`.
3. Rezultat operacji wyświetl (obraz i jego histogram).
4. Czy ilość "widocznych" szczegółów uległa zmianie?

In [ ]:
if not os.path.exists("hist1.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/03_Histogram/hist1.bmp --no-check-certificate

if not os.path.exists("hist2.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/03_Histogram/hist2.bmp --no-check-certificate

if not os.path.exists("hist3.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/03_Histogram/hist3.bmp --no-check-certificate

if not os.path.exists("hist4.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/03_Histogram/hist4.bmp --no-check-certificate
#img = [cv2.imread(f'hist{i}.bmp', cv2.IMREAD_GRAYSCALE) for i in range(2,5)]
hist1 = cv2.imread('hist1.bmp', cv2.IMREAD_GRAYSCALE)
fig, ax = plt.subplots(1,2)
fig.set_size_inches(12,6)
ax[0].imshow(hist1)
ax[0].set_xticks([])
ax[0].set_yticks([])
ax[0].set_title(f"hist1.bmp")
ax[1].plot(cv2.calcHist([hist1],[0], None, [256] ,[0,255]))
ax[1].set_title(f"hist1.bmp histogram")
## rozciąganie
fig, ax = plt.subplots(1,2)
fig.set_size_inches(12,6)
hist1_wide = np.zeros(hist1.shape,dtype= 'uint8')
cv2.normalize(hist1,hist1_wide, 0,255,cv2.NORM_MINMAX)
ax[0].imshow(hist1_wide)
ax[0].set_xticks([])
ax[0].set_yticks([])
ax[0].set_title(f"normalized hist1.bmp")
ax[1].plot(cv2.calcHist([hist1_wide],[0], None, [256] ,[0,255]))
ax[1].set_title(f"hist1.bmp histogram")

## Wyrównywanie histogramu

<div style="text-align: justify">
Bardziej zaawansowaną metodą jest tzw. wyrównywanie histogramu (ang. *histogram equalization – HE*).
Idea jest następująca: z punktu widzenia lepszego wykorzystania dostępnych poziomów jasności pożądane jest rozciągnięcie "szczytów" histogramu, a~skompresowanie "dolin" tak, aby taka sama liczba pikseli reprezentowana była przez każdy z dostępnych poziomów jasności (a przynjamniej przybliżona).
Warto zwrócić uwagę, że takie przekształcenie powoduje częściową utratę informacji o szczegółach w obszarach "dolin".
Inaczej mówiąc, dążymy do sytuacji, aby histogram był względnie jednostajny.
Operacją, która pozwala wykonać wyrównywanie histogramu, jest przekształcenie LUT z funkcją przejścia w postaci histogramu skumulowanego danego obrazu.</div><br>

<div style="text-align: justify">
Histogram skumulowany to funkcja obliczona na podstawie histogramu.
Jej pierwszy element to liczba pikseli o odcieniu $0$.
Kolejne wartości to liczba pikseli o odcieniach od $0$ do $n$.</div>

\begin{equation}
C(n) = \sum_{i=0}^{n} h(i)
\end{equation}

<div style="text-align: justify">
Jeżeli histogram jest w postaci znormalizowanej (gęstość rozkładu prawdopodobieństwa) to histogram skumulowany stanowi dystrybuantę rozkładu prawdopodobieństwa.</div><br>

1. Wyznacz histogram skumulowany dla obrazu *hist1.bmp*.
W tym celu wykorzystaj metodę `cumsum` dla histogramu wczytanego obrazu.
Nie przyjmuje ona żadnych argumentów, a zwraca skumulowane wartości tablicy, dla której została użyta.
Histogram należy wyliczyć dla **obrazka wejściowego**, a nie dla wyniku rozciągania.
2. Histogram skumulowany wyświetl razem z histogramem zwykłym na jednym wykresie (nie obok siebie).
Na potrzeby wyświetlenia przeskaluj histogram skumulowany tak, by miał taką samą wartość maksymalną jak zwykły histogram.
W tym celu wykorzystaj metodę `max`.
3. Wyświetlenie kilku linii na jednym wykresie może być zrealizowane w następujący sposób:
        figHistCum, axsHistCum = plt.subplots()

        axsHistCum.plot(HHist)
        axsHistCum.plot(CHistNorm)
        axsHistCum.grid()
4. Teraz zaimplementuj klasyczny algorytm wyrównywania histogramu.
Wykorzystać należy obliczony histogram skumulowany.
Należy go przeskalować w taki sposób aby na jego podstawie zrealizować przekształcenie LUT, czyli do zakresu 0 - 255.

>Uwaga. Opisany algorytm wyrównywania histogramu jest wersją uproszczoną.
>W wersji pełnej należy podczas skalowania tablicy przekodowań LUT pominąć elementy równe *0*.
>
>W tym celu należy wykorzystać funkcje `np.ma.masked_equal` i `np.ma.filled`.
>Pierwsza służy do ukrywania elementów tablicy, natomiast druga zamienia ukryte elementy na podaną wartość.
>W tym przypadku elementem ukrywanym i wpisywaną wartością byłoby *0*.

5. Na kolejnym rysunku wyświetl obrazek po przekształceniu, jego histogram oraz histogram skumulowany.
Co szczególnego można powiedzieć o jego histogramie i histogramie skumulowanym?
6. W bibliotece *OpenCV* dostępna jest funkcja wykonująca wyrównywanie histogramu `cv2.equalizeHist`.
Jej argumentem jest obraz, którego histogram zostanie wyrównany. Zwraca natomiast obraz wynikowy.
Na kolejnym rysunku wyświetl wynik funkcji, jego histogram oraz histogram skumulowany.
7. W wykorzystywanej bibliotece zaimplementowana jest również metoda adaptacyjnego wyrównywania histogramu algorytmem CLAHE (ang. *Contrast Limited Adaptive Histogram Equalization*}.
   Kilka słów wyjaśnienia.
   Wadą poznanej metody HE jest jej "globalność" rozumiana jako nieuwzględnianie lokalnych właściwości obrazu.
   Dlatego też powstała metoda adaptacyjnego wyrównywania histogramu (AHE).
   Jest ona spotykana w dwóch wariantach:
   - dla każdego piksela niezależnie, w pewnym jego otoczeniu, wyznaczany jest histogram i przeprowadzane wyrównywanie.
     Jak nietrudno się domyślić rozwiązanie jest dość kosztowne obliczeniowo.
   - obraz wejściowy dzielony jest na nienachodzące na siebie prostokątne okna.
     W każdym z okien obliczany jest histogram i przeprowadzane jest wyrównywanie.
     W celu eliminacji błędów na granicy okien, stosuje się interpolację.

   Metoda AHE ma jednak tą wadę, że w obszarach jednorodnych wzmacnia szum.
   Dlatego też zaproponowano rozwiązanie CLAHE, które zakłada ograniczenie kontrastu (CL).
   W metodzie definiuje się maksymalną wartość danego przedziału histogramu (próg ograniczenia kontrastu).
   Piksele, które przekraczają próg są następnie równomiernie rozdzielane pomiędzy poszczególne przedziały.
   Bardziej szczegółowy opis obu metod dostępny jest na [Wikipedii](https://en.wikipedia.org/wiki/Adaptive_histogram_equalization).

8.W celu użycia algorytmu należy wywołać funkcję `cv2.createCLAHE`.
    - Pierwszym parametrem jest próg ograniczenia kontrastu.
    - Drugi parametr definiuje na ile prostokątów zostanie podzielony obraz w rzęch i kolumnach.
    - Zwracany jest zainicjalizowany *smart pointer* do klasy `cv::CLAHE`.
9. Aby wykonać wyrównywanie należy użyć metody `apply`.
Jej argumentem jest obraz wejściowy. Zwracany jest obraz o zmodyfikowanym histogramie.
10. Przetestuj różne parametry algorytmu CLAHE.
11. W kolejnym etapie należy przetestować operacje (rozciąganie, wyrównywanie (HE) i adaptacyjne wyrównywanie CLAHE)  na histogramie dla obrazów rzeczywistych. *hist2.bmp*, *hist3.bmp*, *hist4.jpg*.
W jednym oknie wyświetl: obraz oryginalny, rozciąganie, wyrównywanie HE oraz wyrównywanie CLAHE.

In [ ]:
hist_dystr=cv2.calcHist([hist1],[0], None, [256] ,[0,256]).cumsum()#np.linspace(0,255,len(hist_dystr))
plt.plot(hist_dystr*max(cv2.calcHist([hist1],[0], None, [256] ,[0,255]))/max(hist_dystr) )
plt.plot(cv2.calcHist([hist1],[0], None, [256] ,[0,255]))
plt.show()

In [ ]:
def hist_eqa(hist_dystr : np.array):
    i = 0
    while hist_dystr[i]==0:
        i+=1
    first_non_zero = hist_dystr[i]
    lUT = np.array([(255)*(hist_dystr[j] - first_non_zero)/(1-first_non_zero) for j in range(0,256)],dtype = 'uint8')
    return lUT

img_2 = cv2.LUT(hist1, hist_eqa(hist_dystr/max(hist_dystr)))
hist_2 = cv2.calcHist([img_2],[0], None, [256] ,[0,256])
hist_dystr_2=hist_2.cumsum()

fig, ax = plt.subplots(1,3)
fig.set_size_inches(16,6)

cv2.normalize(hist1,hist1_wide, 0,255,cv2.NORM_MINMAX)
ax[0].imshow(img_2)
ax[0].set_xticks([])
ax[0].set_yticks([])
ax[0].set_title(f"hist1.bmp")
ax[1].plot(hist_2)
ax[1].set_title(f"hist1.bmp histogram")
ax[2].plot(hist_dystr_2)
ax[2].set_title(f"hist1.bmp histogram cumsum")

In [ ]:
fig, ax = plt.subplots(1,3)
fig.set_size_inches(16,6)
img_3 = cv2.equalizeHist(hist1)
ax[0].imshow(img_3)
ax[0].set_xticks([])
ax[0].set_yticks([])
ax[0].set_title(f"hist1.bmp")
ax[1].plot(cv2.calcHist([img_2],[0], None, [256] ,[0,256]))
ax[1].set_title(f"hist1.bmp histogram")
ax[2].plot(cv2.calcHist([img_2],[0], None, [256] ,[0,256]).cumsum())
ax[2].set_title(f"hist1.bmp histogram cumsum")

In [ ]:
clah = cv2.createCLAHE()
fig, ax  = plt.subplots(1,3)
fig.set_size_inches(16,6)
ax[0].imshow(clah.apply(hist1))
ax[0].set_xticks([])
ax[0].set_yticks([])
ax[0].set_title("CLAHE image")
ax[1].set_title("histogram")
ax[2].set_title("empiric dystrybuant")
ax[1].plot(cv2.calcHist([clah.apply(hist1)],[0], None, [256] ,[0,256]))
ax[2].plot(cv2.calcHist([clah.apply(hist1)],[0], None, [256] ,[0,256]).cumsum())
plt.show()

clah = cv2.createCLAHE(clipLimit=4.0,tileGridSize=(10,8))
fig, ax  = plt.subplots(1,3)
fig.set_size_inches(16,6)
ax[0].imshow(clah.apply(hist1))
ax[0].set_xticks([])
ax[0].set_yticks([])
ax[0].set_title("CLAHE image")
ax[1].set_title("histogram")
ax[2].set_title("empiric dystrybuant")
ax[1].plot(cv2.calcHist([clah.apply(hist1)],[0], None, [256] ,[0,256]))
ax[2].plot(cv2.calcHist([clah.apply(hist1)],[0], None, [256] ,[0,256]).cumsum())
plt.show()

In [ ]:
def clahe(hist1, clp, tileGsize,title):
    clah = cv2.createCLAHE(clipLimit=clp,tileGridSize=tileGsize)
    fig, ax  = plt.subplots(1,3)
    fig.set_size_inches(16,6)
    ax[0].imshow(clah.apply(hist1))
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    ax[0].set_title(title)
    ax[1].set_title("histogram")
    ax[2].set_title("empiric dystrybuant")
    ax[1].plot(cv2.calcHist([clah.apply(hist1)],[0], None, [256] ,[0,256]))
    ax[2].plot(cv2.calcHist([clah.apply(hist1)],[0], None, [256] ,[0,256]).cumsum())
    plt.show()
img = [cv2.imread(f'hist{i}.bmp', cv2.IMREAD_GRAYSCALE) for i in range(2,5)]
for i in img:
    fig, ax  = plt.subplots(1,3)
    fig.set_size_inches(16,6)
    ax[0].imshow(i)
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    ax[0].set_title("orginal image")
    ax[1].set_title("histogram")
    ax[2].set_title("empiric dystrybuant")
    ax[1].plot(cv2.calcHist([i],[0], None, [256] ,[0,256]))
    ax[2].plot(cv2.calcHist([i],[0], None, [256] ,[0,256]).cumsum())
    plt.show()
    for j in [[8,(4,4)],[20,(8,8)]]:
        clahe(i,j[0],j[1], f"Clahe : clipLimit={j[0]} \n tileGridSize={j[1]}")


## Histogram dla obrazów kolorowych i jego wyrównywanie

1. Wczytaj obraz *lenaRGB.bmp*.
2. Wykonaj konwersję przestrzeni barw z BGR do RGB.
3. Wyświetl wczytany obraz oraz histogram dla każdej składowej przestrzeni barw.
W tym celu można użyć drugiego argumentu wykorzystywanej funkcji (numer kanału).
4. Wykonaj wyrównywanie dla każdej składowej obrazu.
Połącz otrzymane składowe w nowy obraz i wyświetl go.
Jaka jest zasadnicza wada takiego podejścia?
5. Przekształć obraz wejściowy do przestrzeni HSV (flaga `cv2.COLOR_BGR2HSV`).
Wyświetl histogramy poszczególnych składowych.
Manipulacji dokonujemy na składowej odpowiadającej za jasność, czyli V.
Wykonaj wyrównywanie histogramu dla tej składowej.
Dokonaj podmiany składowej V i wyświetl rezultat operacji.
Uprzednio przeprowadź konwersję HSV->RGB (flaga `cv2.COLOR_HSV2RGB`).
6. Wykonaj te same operacje dla obrazu *jezioro.jpg*.

In [ ]:
if not os.path.exists("lenaRGB.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/03_Histogram/lenaRGB.bmp --no-check-certificate

if not os.path.exists("jezioro.jpg") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/03_Histogram/jezioro.jpg --no-check-certificate
lenabgr = cv2.imread('lenaRGB.bmp')
lenaRGB = cv2.cvtColor(lenabgr, cv2.COLOR_BGR2RGB)
lenaHSV = cv2.cvtColor(lenaRGB, cv2.COLOR_RGB2HSV)

jeziorobgr = cv2.imread('jezioro.jpg').astype('uint8')
jezioroRGB = cv2.cvtColor(jeziorobgr, cv2.COLOR_BGR2RGB).astype('uint8')
jezioroHSV = cv2.cvtColor(jezioroRGB, cv2.COLOR_RGB2HSV).astype('uint8')

plt.imshow(lenaRGB)
plt.xticks([])
plt.yticks([])
plt.show()
def RGBhist(img,title):
    fig, ax  = plt.subplots(2,2)
    fig.set_size_inches(12,8)
    ax[0][0].imshow(img)
    ax[0][0].set_xticks([])
    ax[0][0].set_yticks([])
    ax[0][0].set_title(title)
    ax[1][0].set_title("histogram Green")
    ax[0][1].set_title("histogram Red")
    ax[1][0].plot(cv2.calcHist([img],[1], None, [256] ,[0,256]))
    ax[0][1].plot(cv2.calcHist([img],[0], None, [256] ,[0,256]))
    ax[1][1].set_title("histogram Blue")
    ax[1][1].plot(cv2.calcHist([img],[2], None, [256] ,[0,256]))
    plt.show()

def equaRGB(img):
    dystr_red, dystr_green, dystr_blue = cv2.calcHist([img],[0], None, [256] ,[0,256]),cv2.calcHist([img],[1], None, [256] ,[0,256]),cv2.calcHist([img],[2], None, [256] ,[0,256])
    img_red , img_green, img_blue = cv2.LUT(img[:,:,0], hist_eqa(dystr_red.cumsum()/max(dystr_red.cumsum()))),cv2.LUT(img[:,:,1], hist_eqa(dystr_green.cumsum()/max(dystr_green.cumsum()))),cv2.LUT(img[:,:,2], hist_eqa(dystr_blue.cumsum()/max(dystr_blue.cumsum())))
    img_reconstructed = np.zeros([img.shape[0],img.shape[1],3])
    img_reconstructed[:,:,0] = img_red
    img_reconstructed[:,:,1] = img_green
    img_reconstructed[:,:,2] = img_blue
    img_reconstructed = cv2.normalize(img_reconstructed,None,0,255,cv2.NORM_MINMAX).astype('uint8')
    return img_reconstructed

def equaV(img):
    dystr_H, dystr_S, dystr_V = cv2.calcHist([img],[0], None, [256] ,[0,256]),cv2.calcHist([img],[1], None, [256] ,[0,256]),cv2.calcHist([img],[2], None, [256] ,[0,256])
    img_H , img_S, img_V = img[:,:,0],img[:,:,1],cv2.LUT(img[:,:,2], hist_eqa(dystr_V.cumsum()/max(dystr_V.cumsum())))
    img_reconstructed = np.zeros([img.shape[0],img.shape[1],3])
    img_reconstructed[:,:,0] = img_H
    img_reconstructed[:,:,1] = img_S
    img_reconstructed[:,:,2] = img_V
    img_reconstructed = cv2.normalize(img_reconstructed,None,0,255,cv2.NORM_MINMAX).astype('uint8')
    img_reconstructed = cv2.cvtColor(img_reconstructed, cv2.COLOR_HSV2RGB)
    img_reconstructed = cv2.normalize(img_reconstructed,None,0,255,cv2.NORM_MINMAX).astype('uint8')
    return img_reconstructed


RGBhist(lenaRGB,"orginal image")
RGBhist(equaRGB(lenaRGB),"obraz wyrównowarzony")
RGBhist(equaV(lenaHSV),"obraz wyrównowarzony względem V")

RGBhist(jezioroRGB,"orginal image")
RGBhist(equaRGB(jezioroRGB),"obraz wyrównowarzony")
RGBhist(equaV(jezioroHSV),"obraz wyrównowarzony względem V")
